In [ ]:
import pandas as pd
import mysql.connector
import datetime
import streamlit as st
from database import (
    initialize_db, add_provider, add_receiver, add_listing, add_claim,
    load_csv_data, analysis_queries
)

In [ ]:
#Load data from csv file
providers =pd.read_csv("providers_data.csv")
receivers =pd.read_csv("receivers_data.csv")
food_listings =pd.read_csv("food_listings_data.csv")
claims =pd.read_csv("claims_data.csv")

In [ ]:

providers


In [ ]:
receivers

In [ ]:
food_listings

In [ ]:
claims

In [ ]:
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Maha2301##",
    # database="food_wastage"
)
cursor = conn.cursor()

cursor.execute("CREATE DATABASE IF NOT EXISTS food_data")
print ("MySQL Database created successfully")

In [ ]:
cursor.execute("USE food_data")
cursor.execute("DROP TABLE IF EXISTS claims")
cursor.execute("DROP TABLE IF EXISTS food_listings")
cursor.execute("DROP TABLE IF EXISTS receivers")
cursor.execute("DROP TABLE IF EXISTS providers")

conn.commit()

In [ ]:


cursor.execute("USE food_data")


cursor.execute("""
CREATE TABLE IF NOT EXISTS providers (
Provider_ID INT PRIMARY KEY,
Name VARCHAR(255),
Type VARCHAR(255),
Address TEXT,
City VARCHAR(100),
Contact VARCHAR(50))""")
conn.commit()

cursor.execute("""
CREATE TABLE IF NOT EXISTS receivers (
    Receiver_ID INT PRIMARY KEY,
    Name VARCHAR(255),
    Type VARCHAR(100),
    City VARCHAR(100),
    Contact VARCHAR(50)
)
""")
conn.commit()

cursor.execute("""
CREATE TABLE IF NOT EXISTS food_listings (
    Food_ID INT PRIMARY KEY,
    Food_Name VARCHAR(255),
    Quantity INT,
    Expiry_Date DATE,
    Provider_ID INT,
    Provider_Type VARCHAR(100),
    Location VARCHAR(255),
    Food_Type VARCHAR(100),
    Meal_Type VARCHAR(100),
    FOREIGN KEY (Provider_ID) REFERENCES providers(Provider_ID)
)
""")
conn.commit()


cursor.execute("""
CREATE TABLE IF NOT EXISTS claims (
    Claim_ID INT PRIMARY KEY,
    Food_ID INT,
    Receiver_ID INT,
    Status VARCHAR(100),
    Timestamp DATETIME,
    FOREIGN KEY (Food_ID) REFERENCES food_listings(Food_ID),
    FOREIGN KEY (Receiver_ID) REFERENCES receivers(Receiver_ID)
)
""")
conn.commit()


#Insrt data using iterrows();

for index, row in providers.iterrows():
    cursor.execute("""
    INSERT INTO providers (Provider_ID, Name, Type, Address, City, Contact)
    VALUES (%s, %s, %s, %s, %s, %s)""",tuple(row))



for index, row in receivers.iterrows():
    cursor.execute("""
    INSERT INTO receivers (Receiver_ID, Name, Type, City, Contact)
    VALUES (%s, %s, %s, %s, %s)
    """, tuple(row))
conn.commit()




for index, row in food_listings.iterrows():
    # Convert Expiry_Date to Python date object
    expiry_date = pd.to_datetime(row['Expiry_Date']).date()
    cursor.execute("""
    INSERT INTO food_listings (
        Food_ID, Food_Name, Quantity, Expiry_Date, Provider_ID,
        Provider_Type, Location, Food_Type, Meal_Type
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, (
        row['Food_ID'], row['Food_Name'], row['Quantity'], expiry_date,
        row['Provider_ID'], row['Provider_Type'], row['Location'],
        row['Food_Type'], row['Meal_Type']
    ))
conn.commit()


for index, row in claims.iterrows():
    timestamp = pd.to_datetime(row['Timestamp'])
    cursor.execute("""
    INSERT INTO claims (Claim_ID, Food_ID, Receiver_ID, Status, Timestamp)
    VALUES (%s, %s, %s, %s, %s)
    """, (
        row['Claim_ID'], row['Food_ID'], row['Receiver_ID'],
        row['Status'], timestamp
    ))
conn.commit()


In [ ]:
#1 SQL query to get How many food providers and receivers are there in each city?

query=""" SELECT City, 
       COUNT(DISTINCT Provider_ID) AS Num_Providers, 
       COUNT(DISTINCT Receiver_ID) AS Num_Receivers
FROM (
    SELECT City, Provider_ID, NULL AS Receiver_ID FROM providers
    UNION ALL
    SELECT City, NULL, Receiver_ID FROM receivers
) AS combined
GROUP BY City"""

cursor.execute(query)
result=cursor.fetchall() 

result

In [ ]:
#2 SQL query to get Which type of food provider contributes the most food?

query=""" SELECT Provider_Type, 
       COUNT(*) AS Num_Food_Items,
       SUM(Quantity) AS Total_Quantity
FROM food_listings
GROUP BY Provider_Type
ORDER BY Total_Quantity DESC
LIMIT 1"""

cursor.execute(query)
result=cursor.fetchall() 

result

In [ ]:
#3 SQL query to get What is the contact information of food providers in a specific city?

query = """
SELECT Name, Contact, Address
FROM providers
WHERE City IN ('New Jessica')
"""

cursor.execute(query)
result=cursor.fetchall() 

result

In [ ]:

#4 SQL query to get Which receivers have claimed the most food?

query=""" SELECT r.Receiver_ID, r.Name, COUNT(c.Claim_ID) AS Num_Claims
FROM claims c
JOIN receivers r ON c.Receiver_ID = r.Receiver_ID
GROUP BY r.Receiver_ID
ORDER BY Num_Claims DESC
LIMIT 5"""

cursor.execute(query)
result=cursor.fetchall() 

result

In [ ]:
#5 SQL query to get What is the total quantity of food available from all providers?

query=""" SELECT SUM(Quantity) AS Total_Available_Quantity
FROM food_listings"""

cursor.execute(query)
result=cursor.fetchone() 
result

In [ ]:
#6 SQL query to get Which city has the highest number of food listings?

query=""" SELECT p.City, COUNT(f.Food_ID) AS Num_Listings
FROM food_listings f
JOIN providers p ON f.Provider_ID = p.Provider_ID
GROUP BY p.City
ORDER BY Num_Listings DESC
LIMIT 1"""

cursor.execute(query)
result=cursor.fetchone() 
result


In [ ]:
#7 SQL query to get What are the most commonly available food types?

query=""" SELECT Food_Type, COUNT(*) AS Frequency
FROM food_listings
GROUP BY Food_Type
ORDER BY Frequency DESC"""

cursor.execute(query)
result=cursor.fetchall() 

result

In [ ]:
#8 SQL query to get How many food claims have been made for each food item?

query=""" SELECT f.Food_ID, f.Food_Name, COUNT(c.Claim_ID) AS Num_Claims
FROM claims c
JOIN food_listings f ON c.Food_ID = f.Food_ID
GROUP BY f.Food_ID, f.Food_Name
ORDER BY Num_Claims DESC"""

cursor.execute(query)
result=cursor.fetchall() 

result

In [ ]:
#9 SQL query to get  Which provider has had the highest number of successful food claims?

query=""" SELECT p.Provider_ID, p.Name, COUNT(*) AS Successful_Claims
FROM claims c
JOIN food_listings f ON c.Food_ID = f.Food_ID
JOIN providers p ON f.Provider_ID = p.Provider_ID
WHERE c.Status = 'Completed'
GROUP BY p.Provider_ID, p.Name
ORDER BY Successful_Claims DESC
LIMIT 1"""

cursor.execute(query)
result=cursor.fetchall() 

result

In [ ]:
#10 SQL query to get What percentage of food claims are completed vs. pending vs. canceled?

query=""" SELECT Status,
       COUNT(*) AS Total,
       ROUND(COUNT(*) * 100 / (SELECT COUNT(*) FROM claims), 2) AS Percentage
FROM claims
GROUP BY Status;
"""

cursor.execute(query)
result=cursor.fetchall() 

result

In [ ]:

#11 SQL query to get What is the average quantity of food claimed per receiver?
#Assuming each claim represents one food item:

query=""" SELECT r.Receiver_ID, r.Name,
       ROUND(AVG(f.Quantity), 2) AS Avg_Claimed_Quantity
FROM claims c
JOIN receivers r ON c.Receiver_ID = r.Receiver_ID
JOIN food_listings f ON c.Food_ID = f.Food_ID
GROUP BY r.Receiver_ID, r.Name
ORDER BY Avg_Claimed_Quantity DESC;
"""

cursor.execute(query)
result=cursor.fetchall() 

result

In [ ]:
#12 SQL query to get Which meal type is claimed the most?

query=""" SELECT f.Meal_Type, COUNT(c.Claim_ID) AS Num_Claims
FROM claims c
JOIN food_listings f ON c.Food_ID = f.Food_ID
GROUP BY f.Meal_Type
ORDER BY Num_Claims DESC
LIMIT 1"""

cursor.execute(query)
result=cursor.fetchall() 

result

In [ ]:
#13 SQL query to get What is the total quantity of food donated by each provider?

query=""" SELECT p.Provider_ID, p.Name, SUM(f.Quantity) AS Total_Donated
FROM food_listings f
JOIN providers p ON f.Provider_ID = p.Provider_ID
GROUP BY p.Provider_ID, p.Name
ORDER BY Total_Donated DESC"""

cursor.execute(query)
result=cursor.fetchall() 

result

In [ ]:
#14 SQL query to get Which providers serve multiple cities?


query="""SELECT Name, COUNT(DISTINCT City) AS Cities_Served
FROM providers
GROUP BY Provider_ID, Name
HAVING Cities_Served > 1"""

cursor.execute(query)
result=cursor.fetchall() 

result

In [ ]:
#15 SQL query to get Which cities have no food listings

query=""" SELECT DISTINCT City
FROM providers
WHERE City NOT IN (
  SELECT DISTINCT City 
  FROM providers p
  JOIN food_listings f USING (Provider_ID)
)"""

cursor.execute(query)
result=cursor.fetchall() 

result

In [ ]:
#16 SQL query to get What is the Food type distribution by city

query=""" SELECT p.City, f.Food_Type, COUNT(*) AS Count
FROM food_listings f
JOIN providers p USING (Provider_ID)
GROUP BY p.City, f.Food_Type"""

cursor.execute(query)
result=cursor.fetchall() 

result

In [ ]:
#17 SQL query to get What is the Number of listings added per month?

query="""SELECT DATE_FORMAT(Timestamp, '%Y-%m') AS Month, COUNT(*) AS Listings_Count
FROM food_listings fl
JOIN claims c ON fl.Food_ID = c.Food_ID
GROUP BY Month"""

cursor.execute(query)
result=cursor.fetchall() 

result

In [ ]:
#18 SQL query to get What is the Number of claims per month?

query="""SELECT DATE_FORMAT(Timestamp, '%Y-%m') AS Month, COUNT(*) AS Claims_Count
FROM claims
GROUP BY Month
ORDER BY Month"""

cursor.execute(query)
result=cursor.fetchall() 

result

In [ ]:
#19 SQL query to get What is the Claims in last 7 days?

query=""" SELECT COUNT(*) AS Recent_Claims
FROM claims
WHERE Timestamp >= DATE_SUB(NOW(), INTERVAL 7 DAY)"""

cursor.execute(query)
result=cursor.fetchall() 

result

In [ ]:
#20 SQL query to get What is the Food items expiring today?

query=""" SELECT * FROM food_listings WHERE Expiry_Date = CURDATE()"""

cursor.execute(query)
result=cursor.fetchall() 

result

In [ ]:
#21 SQL query to get who are the Receivers who never claimed?

query=""" SELECT Receiver_ID, Name
FROM receivers
WHERE Receiver_ID NOT IN (SELECT DISTINCT Receiver_ID FROM claims)"""

cursor.execute(query)
result=cursor.fetchall() 

result

In [ ]:
#22 SQL query to get who are the Providers with no listings?

query="""SELECT Provider_ID, Name
FROM providers
WHERE Provider_ID NOT IN (SELECT DISTINCT Provider_ID FROM food_listings)"""

cursor.execute(query)
result=cursor.fetchall() 

result

In [ ]:
#23 SQL query to get What is the Best time window for food claims (hour of day)?

query=""" SELECT HOUR(Timestamp) AS Claim_Hour, COUNT(*) AS Num_Claims
FROM claims
GROUP BY Claim_Hour
ORDER BY Num_Claims DESC"""

cursor.execute(query)
result=cursor.fetchall() 

result

In [ ]:
cursor.close()
conn.close()

In [ ]:
import streamlit as st
from database import *

st.title("🍲 Local Food Wastage Management System")

menu = ["Home", "View Data", "CRUD Operations", "Analysis"]
choice = st.sidebar.selectbox("Menu", menu)

if choice == "Home":
    st.subheader("Welcome to the Food Wastage Management Platform")
    st.markdown("""
        - 📋 List surplus food
        - 🤝 Claim available food
        - 📊 Visualize trends
        - 🔍 Filter by location, provider, type
    """)

elif choice == "View Data":
    table = st.selectbox("Select Table", ["providers", "receivers", "food_listings", "claims"])
    df = run_query(f"SELECT * FROM {table}")
    st.dataframe(df)

elif choice == "CRUD Operations":
    st.subheader("Add a New Food Listing")
    with st.form("add_form"):
        food_id = st.number_input("Food ID", min_value=1)
        food_name = st.text_input("Food Name")
        quantity = st.number_input("Quantity", min_value=1)
        expiry_date = st.date_input("Expiry Date")
        provider_id = st.number_input("Provider ID", min_value=1)
        provider_type = st.text_input("Provider Type")
        location = st.text_input("Location")
        food_type = st.selectbox("Food Type", ["Vegetarian", "Non-Vegetarian", "Vegan"])
        meal_type = st.selectbox("Meal Type", ["Breakfast", "Lunch", "Dinner", "Snacks"])
        submitted = st.form_submit_button("Add")
        if submitted:
            add_food_listing((
                food_id, food_name, quantity, expiry_date,
                provider_id, provider_type, location, food_type, meal_type
            ))
            st.success(f"✅ Added food item: {food_name}")

elif choice == "Analysis":
    st.subheader("📊 Analysis & Insights")

    if st.button("Show Providers & Receivers Per City"):
        df = providers_receivers_per_city()
        st.table(df)

    if st.button("Show Provider Type with Most Contributions"):
        df = most_food_provider_type()
        st.table(df)